In [48]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
import copy

1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).

2. Проведите предобработку.

3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

Task 1

In [49]:
data_train = pd.read_csv('salary-train.csv')
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [50]:
data_test = pd.read_csv('salary-test-mini.csv')

In [51]:
X_train = copy.deepcopy(data_train[['FullDescription', 'LocationNormalized', 'ContractTime']])
y_train = data_train[['SalaryNormalized']]
X_test = copy.deepcopy(data_test[['FullDescription', 'LocationNormalized', 'ContractTime']])

Task 2

In [52]:
#convert to lower register
X_train['FullDescription'] = X_train['FullDescription'].apply(lambda x: x.lower())
X_train['LocationNormalized'] = X_train['LocationNormalized'].apply(lambda x: x.lower())
X_test['FullDescription'] = X_test['FullDescription'].apply(lambda x: x.lower())
X_test['LocationNormalized'] = X_test['LocationNormalized'].apply(lambda x: x.lower())

In [53]:
#replace everything except numbers and letters with spaces to make separation of words easier
X_train = X_train.replace('[^a-zA-Z0-9]', ' ', regex = True)
X_test = X_test.replace('[^a-zA-Z0-9]', ' ', regex = True)

In [54]:
X_train['LocationNormalized'].fillna('nan', inplace=True)
X_train['ContractTime'].fillna('nan', inplace=True)

In [55]:
#leave only words that appear at least 5 times in objects
vectorizer = TfidfVectorizer(min_df = 5)
X_train_vec = vectorizer.fit_transform(X_train['FullDescription'])
X_test_vec = vectorizer.transform(X_test['FullDescription'])

In [56]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(X_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(X_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [58]:
final_train = hstack([X_train_vec, X_train_categ])
final_test = hstack([X_test_vec, X_test_categ])

Task 3

In [59]:
ridge = Ridge(alpha = 1, random_state = 241)
ridge.fit(final_train, y_train)

Ridge(alpha=1, random_state=241)

Task 4

In [62]:
ridge.predict(final_test)

array([[56574.52118845],
       [37138.23965235]])